In [4]:
import warnings
warnings.filterwarnings('ignore')
import ipywidgets as widgets
import anywidget
import traitlets
import jupyter
import tweet_browser as tb
from matplotlib import pyplot as plt
from IPython.display import display, Javascript
import pandas as pd
import numpy as np
import io
import math
import time
import json
import time
from datetime import datetime
import plotly.express as px
import plotly.graph_objects as go
from custom_widgets import *

# TWEETS_PER_PAGE = 20
# DEBUG_MODE = True
# # JUPYTER_FILE_PATH = "../tree/images/"
# JUPYTER_FILE_PATH = "images/"

out = widgets.Output()

fileUp = widgets.widgets.FileUpload(
    accept='.csv, .txt, .xls, .tsv',
    multiple=False,
    description='Change Dataset'
)

dummyEl = DummyElement()
def startSession(file):
    if file['type'] == 'xls':
        data = pd.read_excel(io.BytesIO(file.content))
    else:
        data = pd.read_csv(io.BytesIO(file.content))
    s = tb.Session(data, False)
    dummyEl.fileName = file['name']
    browser = Browser(s, out)

def autoStartSession(fileName):
    data = tb.parse_data(fileName)
    # s = tb.Session(data, False)
    s = tb.Session(data, False, embeddings=pd.read_csv("allCensus_sample_embeddings.csv", encoding = "utf-8", index_col=0))
    dummyEl.fileName = fileName
    browser = Browser(s, out)

def selectColumns (row, colHeaders: list):
    result = []
    for j in colHeaders:
        result.append(row[s.headerDict[j]])
    return result
    
class Browser:
    def __init__(self, s, out):
        self.s = s
        dummyEl.size = s.length
        dummyEl.observe(self.alertHandler, ["changeSignal"])
        self.screen = "main"
        self.colHeaders = list(s.headerDict.keys())
        self.createWidgets()
        self.search()
        self.resetDisplay()
        # self.history = [StoredSearch()]

### Widgets
    def createWidgets(self):
        self.advancedButton = widgets.Button(description='Click here to enter search query').add_class("long-button")
        self.advancedButton.on_click(self.openSearchMenu)
        self.searchedCriteria = widgets.HTML("SEARCHED CRITERIA", layout=widgets.Layout(padding="0px 32px")).add_class("heading4")
        self.tweetDisplay = TweetDisplay(height = "60vh", displayAddOn = 0, addOnColumnName = "SimilarityScore")
        self.tweetDisplay.observe(self.getTweets, names=["pageNum"])
        self.datasetDisplay = fileUp
        self.sortBar = SortBar()
        self.sortBar.observe(self.getTweets, names=["sortScope", "sortColumn", "sortOrder"])
        self.displaySimilarityScore = ToggleSwitch(label = "Relevance", hidden=1, value=0).add_class("tweet-display-add-on")
        self.displaySimilarityScore.observe(self.toggleSimilarityScore, ["value"])
        # optionsBar = widgets.Box(children = [self.sortBar])
        # optionsBar.layout = widgets.Layout(align_items = "center", justify_content = "space-between", width = "100%")
        # self.searchedKeywords = ParameterDisplay(firstWord = "Searched", secondWord = "Keywords", headers = ["Must Include", "Contain one of", "Exclude"], notFound = 'To enter keywords, click "Search & Filter"')
        # self.appliedFilters = ParameterDisplay(firstWord = "Applied", secondWord = "Filters", headers = ["calendar.svg", "geography.svg", "username.svg", "repost.svg", "weight.svg"], notFound = 'To enter filters, click "Search & Filter"')
        self.advancedBar = widgets.VBox([self.searchedCriteria, self.advancedButton]).add_class("advanced-bar")
        self.sampleTitle = widgets.HTML().add_class("display-count")
        self.sampleSelector = SampleSelector(label="Generate New Sample >")
        self.sampleSelector.observe(self.generateNewSample, names=["changeSignal"])
        sampleTopBar = widgets.HBox([self.sampleTitle, self.sampleSelector], layout=widgets.Layout(justify_content="space-between", flex="0 0"))
        sortingBar = widgets.HBox([self.sortBar, self.displaySimilarityScore], layout=widgets.Layout(flex="0 0"))
        self.randomSelection = widgets.VBox([sampleTopBar, sortingBar, self.tweetDisplay], layout=widgets.Layout(max_height="100%"))

        self.makeCentralityTab()
        self.makeAiSummaryPage()
        self.makeFilterBar()
        self.makeStanceAnalysisPage()
        self.makeTimeSeriesPage()
        
        self.tabs = widgets.Tab(children=[self.randomSelection, self.centralTweetBox, self.summaryTab, self.stanceAnalysis, self.timeSeries], titles=["Random Posts", "Typical Posts", "AI Summary", "Stance Analysis", "Time Series"])
        self.tabs.observe(self.loadTab, names=["selected_index"])
        self.topBar = widgets.HBox([widgets.HTML("Social Media Browser").add_class("title"), self.datasetDisplay]).add_class("top-bar")
        self.mainPage = widgets.VBox([self.topBar, self.advancedBar, widgets.HBox([self.filterBox, self.tabs])])
        # self.mainPage = widgets.VBox([self.paramDisplay, self.tabs])
        self.debugText = widgets.HTML("test")
        self.makeAdvancedPage()

    def makeCentralityTab(self):
        self.typicalSampleTitle = widgets.HTML().add_class("display-count")
        self.displayCentralityScore = ToggleSwitch(label = "Typicality", value=0).add_class("tweet-display-add-on")
        self.displayCentralityScore.observe(self.toggleTypicalityScore, ["value"])
        self.centralTweets = TweetDisplay(height="60vh", displayAddOn=0, addOnColumnName="centrality")
        self.centralTweetBox = widgets.VBox([widgets.HBox([self.typicalSampleTitle, self.displayCentralityScore], layout=widgets.Layout(flex="0 0", margin="0px 0px 16px 0px")), self.centralTweets], layout=widgets.Layout(max_height="100%"))
        
    def makeFilterBar(self):
        self.filterBy = widgets.HTML(value = "Refine Results").add_class("heading5").add_class("medium")
        dateRange = widgets.HTML(value = "Date").add_class("body2").add_class("medium")
        self.fromDate = widgets.DatePicker(description = "From")
        self.toDate = widgets.DatePicker(description = "To")
        minDate = self.s.findMinDate().strftime("%Y-%m-%d")
        maxDate = self.s.findMaxDate().strftime("%Y-%m-%d")
        dummyEl.calendarStart = minDate
        dummyEl.calendarEnd = maxDate
        self.fromDate.add_class("date-constraint") # The script to set the elements attribute is attached to the toggleSwitch widget
        self.toDate.add_class("date-constraint") # This was done for convenience and should be changed later
        self.weightBy = WeightBy()
        self.dateBox = widgets.VBox([dateRange, widgets.HBox([self.fromDate, self.toDate])]).add_class("date-bar")
        self.allowRetweets = ToggleSwitch(label = "Include reposts")
        self.retweets = widgets.VBox([widgets.HTML(value = "Retweets").add_class("body2").add_class("medium"), self.allowRetweets])
        self.geography = SearchBar(header = "Geography", placeholder = "Search")
        self.userName = SearchBar(header = "Username", placeholder = "Search")
        self.mainPageClear = widgets.Button(description='Clear All').add_class("clear-button")
        self.mainPageClear.on_click(self.clearFilters)
        self.mainPageSearch = widgets.Button(description='Apply').add_class("generic-button").add_class("main-page-search")
        self.mainPageSearch.on_click(self.search)
        self.popUpOptions = widgets.HBox([self.mainPageClear, self.mainPageSearch]).add_class("pop-up-options")
        self.userName.observe(self.displayPopUp, names=["value"])
        self.geography.observe(self.displayPopUp, names=["value"])
        self.allowRetweets.observe(self.displayPopUp, names=["value"])
        self.weightBy.observe(self.displayPopUp, names=["value"])
        self.fromDate.observe(self.displayPopUp, names=["value"])
        self.toDate.observe(self.displayPopUp, names=["value"])
        self.mainFilters = widgets.VBox([self.filterBy, self.dateBox, self.retweets, self.geography, self.userName, self.weightBy]).add_class("main-filters")
        self.filterBox = widgets.VBox([self.mainFilters]).add_class("filter-box")

    def makeAdvancedPage(self):
        self.searchButton = widgets.Button(description='Search', icon="search").add_class("generic-button").add_class("search-button")
        self.hiddenButton = widgets.Button()
        self.hiddenButton.add_class("hidden-button") # work around for syncing search when the user still has input in the search bars
        self.hiddenButton.on_click(self.search)
        self.clearButton = widgets.Button(description='Clear All').add_class("clear-button")
        self.clearButton.on_click(self.clearSearch)
        self.bottomBar = widgets.HBox([self.clearButton, self.searchButton, self.hiddenButton], layout = widgets.Layout(justify_content = "flex-end"))
        keyWordSearch = widgets.HTML(value = "Exact Match", layout = widgets.Layout(margin = "0px 0px -8px 0px")).add_class("heading5").add_class("medium")
        self.mustInclude = SearchBar(header = "Must include all", header2="(AND)", placeholder='e.g. “civil null” means each post in the result must contain the word “civil” and “null”')
        self.containOneOf = SearchBar(header = "Must include one of", header2="(OR)", placeholder='e.g. “census penny” means each post in the result must contain either “census” or “penny” or both')
        self.exclude = SearchBar(header = "Must not include", header2="(NOT)", placeholder='e.g. “toxic ban” means none of the posts in the result contains the word “toxic” and “ban”')
        self.semanticSearch = SemanticSearch(placeholder = "e.g. misinformation and miscommunication")
        self.searches = widgets.VBox([widgets.HTML(value = "<b>Search Criteria</b>"), self.semanticSearch, keyWordSearch, self.mustInclude, self.containOneOf, self.exclude])
        self.searches.add_class("search-box")
        self.closeButton = widgets.Button(description = 'X')
        self.closeButton.add_class("close-button")
        self.closeButton.on_click(self.closeSearchMenu)
        # self.advancedBox = widgets.HBox([self.searches, self.filterBox])
        # self.advancedBox.add_class("advanced-box")
        # self.advancedPage = widgets.VBox([self.advancedBox, self.bottomBar])
        self.advancedPage = widgets.VBox([self.closeButton, self.searches, self.bottomBar]).add_class("advanced-page")
    
    def makeAiSummaryPage(self):
        self.loadingPage = LoadingPage(text="Generating AI Summary")
        self.aiSummary = AiSummary()
        self.aiSummary.observe(self.updateAiPageSelect, names=["changeSignal"])
        self.aiTitle = widgets.HTML().add_class("display-count")
        self.pageSelectAi = PageSelect()
        self.pageSelectAi.observe(self.getSummaryTweets, names=["value"])
        self.summaryDisplay = TweetDisplay(height="60vh")
        leftBar = widgets.VBox([widgets.HTML("AI Generated Summary").add_class("heading4").add_class("medium"), self.aiSummary, self.pageSelectAi]).add_class("left-bar")
        self.newSummaryButton = widgets.Button(description="Generate Another Summary").add_class("generic-button").add_class("summary-button")
        self.newSummaryButton.on_click(self.generateNewSummary)
        summaryContent = widgets.HBox([leftBar, widgets.VBox([widgets.HTML("Contributing Posts").add_class("heading4").add_class("medium"), self.summaryDisplay]).add_class("right-bar")]).add_class("summary-tab")
        self.summaryTab = widgets.VBox([self.aiTitle, summaryContent, self.newSummaryButton], layout=widgets.Layout(height="100%"))

    def makeStanceAnalysisPage(self):
        self.stanceAnalysis = StanceAnalysis()
        self.stanceAnalysis.observe(self.showStanceAnalysisResults, names=["pageNumber"])
        self.stanceAnalysis.observe(self.openSearchMenu, names=["changeSignal"])
        
        self.modifyStanceButton = widgets.Button(description="< Modify Stance Annotation", layout=widgets.Layout(margin="0 auto 0 0", flex="0 0")).add_class("clear-button")
        self.modifyStanceButton.on_click(self.backToStancePage)
        self.stanceSampleSelector = SampleSelector(label="New Stance Analysis >")
        self.stanceSampleSelector.observe(self.showStanceAnalysisResults, names=["changeSignal"])
        self.stanceLoadingScreen = LoadingPage(text="Applying Stance Analysis")
        self.stanceTitle = widgets.HTML().add_class("display-count")
        self.stance0CheckBox = widgets.Checkbox(value=True, indent=False).add_class("stance-checkbox0")
        self.stance1CheckBox = widgets.Checkbox(value=True, indent=False).add_class("stance-checkbox1")
        self.stance2CheckBox = widgets.Checkbox(value=True, indent=False).add_class("stance-checkbox2")
        self.stance3CheckBox = widgets.Checkbox(value=True, indent=False).add_class("stance-checkbox3")
        self.defaultStanceCheckBox = widgets.Checkbox(value=True, indent=False, description="System Default - Irrelevant").add_class("stance-checkbox-1")
        self.checkboxes = [self.stance0CheckBox, self.stance1CheckBox, self.stance2CheckBox, self.stance3CheckBox, self.defaultStanceCheckBox]
        for cb in self.checkboxes:
            cb.observe(self.getStanceTweets, names=["value"])
        self.checkboxBar = widgets.HBox(self.checkboxes, layout=widgets.Layout(flex="0 0"))
        self.stanceTopBar = widgets.HBox([self.stanceTitle, self.stanceSampleSelector], layout=widgets.Layout(flex="0 0", justify_content="space-between"))
        self.stanceSortBar = SortBar(columns=["None", "Date", "Geography", "Retweets", "Username", "Stance"], columnNames=["None", "CreatedTime", "State", "Retweets", "SenderScreenName", "stance"])
        self.stanceSortBar.observe(self.getStanceTweets, names=["sortScope", "sortColumn", "sortOrder"])
        self.stanceTweetDisplay = TweetDisplay(displayAddOn=0, colorCode=1)
        self.stanceTweetDisplay.observe(self.getStanceTweets, names=["pageNum"])
        self.stanceTweetDisplay.observe(self.addStanceCorrection, names=["newStanceCorrectionNum"])
        self.stanceCorrections = []
        self.stanceCorrectionNums = []
        self.cancelModification = widgets.Button(description="Cancel").add_class("clear-button")
        self.cancelModification.on_click(self.cancelStanceCorrection)
        self.retrainButton = widgets.Button(description="Update Classification").add_class("generic-button")
        self.retrainButton.on_click(self.showStanceAnalysisResults)
        self.stancePopUp = widgets.HBox([self.cancelModification, self.retrainButton]).add_class("pop-up-options").add_class("stance-pop-up")
        self.stanceAnalysisResults = widgets.VBox([self.modifyStanceButton, self.stanceTopBar, self.checkboxBar, self.stanceSortBar, self.stanceTweetDisplay]).add_class("stance-vbox")
        self.stanceAnalysisPage = self.stanceAnalysis

    def makeTimeSeriesPage(self):
        test = widgets.HTML("test")
        self.timeSeries = widgets.VBox([test])
        
### Control flow
    def resetDisplay(self, b = None):
        out.clear_output(True)
        with out:
            if self.screen == "main":
                display(self.mainPage)
                self.loadTab(None, self.tabs.selected_index)
                self.getTweets()
                if DEBUG_MODE:
                    temp = self.s.currentSet
                    if self.s.currentSet != self.s.base:
                        self.s.back()
                    self.s.currentSet = temp
                    assert(self.s.currentSet.size >= len(self.tweetDisplay.value))
            elif self.screen == "advanced":
                display(self.advancedPage)
            # display(self.debugText)
            display(dummyEl)

    def alertHandler(self, change):
        if dummyEl.userResponse == 1:
            self.search()
            self.loadTab(None, self.loadTab.selected_index)
        dummyEl.userResponse = 2

    def loadTab(self, change, tabNum = None):
        if change != None:
            tabNum = change["new"]
            if self.hasChanges:
                dummyEl.alertTrigger += 1
        if tabNum == 1:
            self.getTypicalPosts()
        elif tabNum == 2:
            self.generateSummary()
        elif tabNum == 4:
            self.generateGraph()

    def openSearchMenu(self, change):
        self.screen = "advanced"
        self.resetDisplay()

    def displayPopUp(self, change):
        self.hasChanges = True
        self.filterBox.children = [self.mainFilters, self.popUpOptions]
### Keyword search          
    def search(self, b=None):
        self.s.currentSet = self.s.base
        self.displaySimilarityScore.hidden = 1
        usedKeywordSearch = False
        for i in range(self.geography.count):
            self.s.filterBy("State", json.loads(self.geography.value)[i].lower().capitalize())
        for i in range(self.userName.count):
            self.s.filterBy("SenderScreenName", json.loads(self.userName.value)[i])
        if(self.fromDate.value != None and self.toDate.value != None):
            self.s.filterDate(self.fromDate.value.strftime('%Y-%m-%d'), self.toDate.value.strftime('%Y-%m-%d'))
        if(self.allowRetweets.value == 0):
            self.s.removeRetweets()
        if(self.semanticSearch.value != ""):
            usedKeywordSearch = True
            self.displaySimilarityScore.hidden = 0
            self.s.semanticSearch(self.semanticSearch.value, float(self.semanticSearch.filterPercent / 100))
        if(self.exclude.count > 0):
            usedKeywordSearch = True
            self.s.exclude(json.loads(self.exclude.value))
        if(self.mustInclude.count > 0):
            usedKeywordSearch = True
            self.s.searchKeyword(json.loads(self.mustInclude.value), False)
        if(self.containOneOf.count > 0):
            usedKeywordSearch = True
            self.s.searchKeyword(json.loads(self.containOneOf.value), True)
        self.toggleSimilarityScore()
        self.filterBox.children = [self.mainFilters]
        self.hasChanges = False
        self.sampleSelector.total = self.s.currentSet.size
        self.stanceSampleSelector.total = self.s.currentSet.size
        self.tweetDisplay.pageNum = 1
        self.currentWorkingSet = self.s.currentSet
        self.aiSummary.rerender = 1
        self.backToStancePage()
        if usedKeywordSearch == False:
            self.stanceAnalysis.pageNumber = -1
        self.updateSearchParams(b)

    def tryGetNewSample(self):
        self.tweetDisplay.pageNum = 1
        sampleSize = self.sampleSelector.value
        if self.s.currentSet.size < self.sampleSelector.value:
            self.sampleSelector.value = -1
        if self.sampleSelector.value == -1:
            sampleSize = self.s.currentSet.size
        
        if DEBUG_MODE:
            assert(sampleSize >= 0)
        if self.weightBy.value == "None":
            self.s.simpleRandomSample(sampleSize)  
        else:
            self.s.weightedSample(sampleSize, self.weightBy.value)
        self.getTweets()        
    
    def generateNewSample(self, b):
        # make sure to call only after a sample has already been generated
        # self.s.back()
        self.s.currentSet = self.currentWorkingSet
        self.tryGetNewSample()
    
    def getTweets(self, change=None):
        dataSet = self.s.getCurrentSubset()
        pageNum = self.tweetDisplay.pageNum
        
        tempArr = []
        sorted = self.getSortedTweets(pageNum)
        if DEBUG_MODE:
            assert(len(sorted) <= 2 * TWEETS_PER_PAGE)
        self.tweetDisplay.maxPage = math.ceil(self.s.currentSet.size / TWEETS_PER_PAGE)
        for i in range(len(sorted)):   
            tempArr.append(sorted.iloc[i].to_json())
        self.sampleTitle.value = "Displaying " + format(self.s.currentSet.size, ',d') + " posts from " + format(self.sampleSelector.total, ',d') + " results"
        self.tweetDisplay.value = tempArr
    
    def getSortedTweets(self, pageNum, currSet=None, sortBar=None):
        if currSet is None:
            currSet = self.s.getCurrentSubset()
        if sortBar is None:
            sortBar = self.sortBar
        ans = currSet
        
        asc = True
        na_pos = "last"
        keyFunc = None
            
        if sortBar.sortColumn == "Username" or sortBar.sortColumn == "SenderScreenName":
            keyFunc = userNameToLower
        if sortBar.sortOrder == "DESC":
            asc = False
            na_pos = "last"
        if sortBar.sortColumn == "None":
            # ans = ans.sample(frac = 1)
            pass
        else:
            ans = ans.sort_values(by=[sortBar.sortColumn], ascending=asc, na_position = na_pos, key=keyFunc)
        ans = ans.iloc[max((pageNum-2) * TWEETS_PER_PAGE, 0) : min(pageNum * TWEETS_PER_PAGE, len(ans))]
        return ans

    def clearSearch(self, change):
        self.semanticSearch.value = ""
        # self.semanticSearch.filterPercent = 50
        self.exclude.value = "[]"
        self.exclude.count = 0
        self.mustInclude.value = "[]"
        self.mustInclude.count = 0
        self.containOneOf.value = "[]"
        self.containOneOf.count = 0
        self.resetDisplay()
        
    def clearFilters(self, change):
        self.geography.value = "[]"
        self.geography.count = 0
        self.userName.value = "[]"
        self.userName.count = 0
        self.fromDate.value = None
        self.toDate.value = None
        self.allowRetweets.value = True
        self.weightBy.value = "None"
        self.filterBox.children = [self.mainFilters]
        self.search()
        # self.resetDisplay()

    def updateSearchParams(self, change):
        searchedString = ""
        if len(self.semanticSearch.value) > 0:
            searchedString += self.semanticSearch.value + " "
        if(self.mustInclude.count > 0):
            searchedString += "(" + ' AND '.join(json.loads(self.mustInclude.value)) + ")"
        if(self.containOneOf.count > 0):
            if len(searchedString) > 0 and searchedString[-1] == ")":
                searchedString += " AND "
            searchedString += "(" + ' OR '.join(json.loads(self.containOneOf.value)) + ")"
        if(self.exclude.count > 0):
            if len(searchedString) > 0 and searchedString[-1] == ")":
                searchedString += " AND "
            searchedString += "NOT (" + ' OR '.join(json.loads(self.exclude.value)) + ")"
        if len(searchedString) > 0:
            self.advancedButton.description = searchedString
        else:
            self.advancedButton.description = "Click here to enter search query"
        newKeywords = ""
        if(self.mustInclude.count > 0):
            newKeywords += ','.join(json.loads(self.mustInclude.value))
        if(self.containOneOf.count > 0):
            if len(temp) > 0:
                newKeywords += ","
            newKeywords += ','.join(json.loads(self.containOneOf.value))
        self.tweetDisplay.keywords = newKeywords
        self.closeSearchMenu(change)

    def closeSearchMenu(self, change):
        self.screen = "main"
        self.resetDisplay()
# Stance analysis logic
    def showStanceAnalysisResults(self, change=None):
        self.s.currentSet = self.currentWorkingSet
        if self.stanceAnalysis.pageNumber <= 0:
            return
        sampleSize = self.stanceSampleSelector.value
        if self.s.currentSet.size < self.stanceSampleSelector.value:
            self.stanceSampleSelector.value = -1
        if self.stanceSampleSelector.value == -1:
            sampleSize = self.s.currentSet.size
        self.s.simpleRandomSample(sampleSize)
        self.tabs.children = [*self.tabs.children[:3], self.stanceLoadingScreen, *self.tabs.children[4:]]
        self.stanceAnalysisResults.children = [self.modifyStanceButton, self.stanceTopBar, self.checkboxBar, self.stanceSortBar, self.stanceTweetDisplay]
        try:
            newCheckboxes = []
            newStances = []
            for i in range(len(self.stanceAnalysis.stances)):
                if self.stanceAnalysis.stances[i] != "":
                    newStances.append(i)
                    self.checkboxes[i].description = "Stance " + str(i+1) + " " + self.stanceAnalysis.stances[i]
                    self.checkboxes[i].value = True
                    newCheckboxes.append(self.checkboxes[i])
            newStances.append(-1)
            self.stanceTweetDisplay.stances = newStances
            newCheckboxes.append(self.checkboxes[-1])
            self.checkboxBar.children = newCheckboxes
            stanceExamples = {}
            for i in range(len(self.stanceAnalysis.examples)):
                if self.stanceAnalysis.examples[i] != "":
                    stanceExamples[self.stanceAnalysis.examples[i]] = i
            for i in range(len(self.stanceCorrections)):
                stanceExamples[self.stanceCorrections[i]] = self.stanceCorrectionNums[i]
                
            self.stanceOutput = self.s.stanceAnalysis(self.stanceAnalysis.topic, self.stanceAnalysis.stances, stanceExamples)
            self.getStanceTweets()
            self.stanceAnalysisPage = self.stanceAnalysisResults
            self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
        except:
            self.stanceAnalysisPage = self.stanceAnalysis
            self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
            self.stanceAnalysis.pageNumber = 0
            with out:
                display(Javascript('alert("Error genearting stance analysis");'))

    def getStanceTweets(self, change=None): # TODO: refactor this into get tweets
        tempDf = self.filterStances()
        
        pageNum = self.stanceTweetDisplay.pageNum
        tempDf["stance"] = tempDf["stance"].replace(-1, None)
        sorted = self.getSortedTweets(pageNum, tempDf, self.stanceSortBar)
        sorted["stance"] = sorted["stance"].fillna(-1)
        self.stanceTweetDisplay.maxPage = math.ceil(len(tempDf) / TWEETS_PER_PAGE)
        tempArr = []
        for i in range(len(sorted)):   
            tempArr.append(sorted.iloc[i].to_json())
        # self.sampleTitle.value = "Displaying " + format(self.s.currentSet.size, ',d') + " posts from " + format(self.sampleSelector.total, ',d') + " results"
        self.stanceTweetDisplay.value = tempArr

    def filterStances(self, change=None, df=None):
        if df is None:
            df = self.stanceOutput.copy(deep=True)
        if self.checkboxes[-1].value == False:
            df = df[df["stance"] != -1]
        for i in range(4):
            if self.checkboxes[i].value == False:
                df = df[df["stance"] != i]
        temp = []
        for i in range(len(df)):
            temp.append(df.iloc[i].to_json())
        # self.stanceTweetDisplay.value = temp
        self.stanceTitle.value = "Displaying " + format(len(df), ',d') + " most relevant tweets from " + format(self.currentWorkingSet.size, ',d') + " results"
        return df

    def addStanceCorrection(self, change):
        self.stanceCorrections.append(self.stanceTweetDisplay.stanceCorrection)
        self.stanceCorrectionNums.append(self.stanceTweetDisplay.newStanceCorrectionNum)
        self.stanceTweetDisplay.stanceCorrection = ""
        self.stanceTweetDisplay.newStanceCorrectionNum = -2
        self.stanceAnalysisResults.children = [self.modifyStanceButton, self.stanceTopBar, self.checkboxBar, self.stanceSortBar, self.stanceTweetDisplay, self.stancePopUp]
    
    def cancelStanceCorrection(self, change=None):
        self.stanceCorrections = []
        self.stanceCorrectionNums = []
        self.stanceAnalysisResults.children = [self.modifyStanceButton, self.stanceTopBar, self.checkboxBar, self.stanceSortBar, self.stanceTweetDisplay]
        self.resetDisplay()
    
    def backToStancePage(self, change=None):
        self.stanceAnalysis.pageNumber = 0
        self.stanceAnalysisPage = self.stanceAnalysis
        self.tabs.children = [*self.tabs.children[:3], self.stanceAnalysisPage, *self.tabs.children[4:]]
        self.cancelStanceCorrection()
### Typical posts logic
    def toggleSimilarityScore(self, change=None):
        self.tweetDisplay.displayAddOn = self.displaySimilarityScore.value
        if self.displaySimilarityScore.hidden > 0:
            self.tweetDisplay.displayAddOn = 0
    
    def toggleTypicalityScore(self, change):
        self.centralTweets.displayAddOn = self.displayCentralityScore.value

    def getTypicalPosts(self, change=None):
        tempArr = []
        self.s.currentSet = self.currentWorkingSet
        result = self.s.getCentral()
        count = min(5, len(result))
        if count == 0:
            self.typicalSampleTitle.visible = False
            self.typicalSampleTitle.value = ""
        else:
            self.typicalSampleTitle.visible = True
            self.typicalSampleTitle.value = "Displaying " + format(count, ",d") + " typical posts from " + format(len(result), ',d') + " results"
        for i in range(count):
            tempArr.append(result.iloc[i].to_json())
        self.centralTweets.value = tempArr
### AI summary logic
    def updateAiPageSelect(self, b):
        self.pageSelectAi.value = self.aiSummary.selected + 1
        self.pageSelectAi.changeSignal += 1
        self.getSummaryTweets(b)

    def generateNewSummary(self, b):
        self.s.currentSet = self.currentWorkingSet
        self.aiSummary.rerender = 1
        self.generateSummary(b)
    
    def getSummaryTweets(self, b):
        pageNum = self.pageSelectAi.value - 1 # convert to 0 indexing
        self.aiSummary.selected = pageNum
        tweets = self.aiSummary.sentenceNums[pageNum]
        ans = self.s.allData.iloc[tweets]
        tempArr = []
        for i in range(len(ans)):
            tempArr.append(ans.iloc[i].to_json())
        self.summaryDisplay.value = tempArr
        self.pageSelectAi.changeSignal += 1
        # self.resetDisplay()
    
    def generateSummary(self, change = None):
        if self.aiSummary.rerender == 0:
            return
        if self.s.currentSet.size > 50:
            self.aiTitle.value = "Summarizing 50 posts sampled from " + format(self.s.currentSet.size, ',d') + " results"
            self.s.simpleRandomSample(50)
        else:
            self.aiTitle.value = "Summarizing " + format(self.s.currentSet.size, ',d') + " posts sampled from " + format(self.s.currentSet.size, ',d') + " results"
        self.tabs.children = [self.randomSelection, self.centralTweetBox, self.loadingPage, self.stanceAnalysisPage]
        summary = self.s.summarize()
        self.tabs.children = [self.randomSelection, self.centralTweetBox, self.summaryTab, self.stanceAnalysisPage]
        strings, tweets, unused = self.s.parseSummary(summary)
        self.aiSummary.value = strings
        self.aiSummary.sentenceNums = tweets
        self.aiSummary.unused = unused
        self.aiSummary.rerender = 0
        self.pageSelectAi.maxPage = len(tweets)
        self.aiSummary.selected = 0
        self.pageSelectAi.value = 1
        self.getSummaryTweets(None)

# Time series logic
    def generateGraph(self, change = None):
        with out:
            print("hi")
            df = self.s.getCurrentSubset()
            data = df.groupby(df["CreatedTime"]).agg({"Message": "count"}).reset_index().rename(columns={"Message": "count"})
            fig = px.line(
                    data,
                    x="CreatedTime",
                    y="count",
                    title=f"testing",
                )
            # htmlString = fig.to_html()
            # htmlString = htmlString[:-16]
            # htmlString = htmlString[56:]
            # test = widgets.HTML(htmlString)
            test = go.FigureWidget(fig)
            self.timeSeries.children = [test]
    
def fileHandler(change):
    startSession(fileUp.value[0])
    
def userNameToLower(input):
    return input.str.lower()

# with out:
#     display(fileUp)

autoStartSession("allCensus_sample.csv")

fileUp.observe(fileHandler, names=["value"])

out

Output()